# A/B Test Analysis

This notebook performs a complete A/B test analysis comparing conversion rates between Variant A (control) and Variant B (treatment).

## Analysis Steps
1. Load A/B test data
2. Perform two-proportion z-test
3. Display test results (z-statistic, p-value, lift, 95% CI)
4. Power analysis with varying MDE
5. Apply decision rule and calculate ROI



In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from abtest import ztest_two_prop, power, load_aggregated_data

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')



## 1. Load Data

Load the A/B test data from CSV file. The data can be in aggregated format (group, success, total) or row-level format (user_id, group, converted).



In [ ]:
# Load aggregated data
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'sample_ab.csv')

try:
    success_a, total_a, success_b, total_b = load_aggregated_data(data_path)
    print("Data loaded successfully!")
    print(f"\nVariant A: {success_a} successes out of {total_a} total")
    print(f"Variant B: {success_b} successes out of {total_b} total")
    print(f"\nConversion rates:")
    print(f"  Variant A: {success_a/total_a:.4f} ({success_a/total_a*100:.2f}%)")
    print(f"  Variant B: {success_b/total_b:.4f} ({success_b/total_b*100:.2f}%)")
except FileNotFoundError:
    print(f"Error: File not found at {data_path}")
    print("Please ensure data/sample_ab.csv exists")



## 2. Statistical Test

Perform a two-proportion z-test to compare conversion rates between variants A and B.



In [ ]:
# Perform two-proportion z-test
results = ztest_two_prop(success_a, total_a, success_b, total_b, alpha=0.05)

print("=" * 60)
print("A/B TEST RESULTS")
print("=" * 60)
print(f"\nZ-statistic: {results['z']:.4f}")
print(f"P-value (two-sided): {results['p']:.6f}")
print(f"\nLift (difference in proportions): {results['lift']:.6f} ({results['lift']*100:.3f} percentage points)")
print(f"\n95% Confidence Interval for difference:")
print(f"  Lower bound: {results['ci'][0]:.6f} ({results['ci'][0]*100:.3f} pp)")
print(f"  Upper bound: {results['ci'][1]:.6f} ({results['ci'][1]*100:.3f} pp)")
print("=" * 60)



## 3. Power Analysis

Analyze statistical power across a range of minimum detectable effect (MDE) sizes to assess test sensitivity.



In [ ]:
# Calculate control rate (variant A)
p_control = success_a / total_a

# Vary MDE from 0.2pp to 5pp (0.002 to 0.05)
mde_values = np.arange(0.002, 0.051, 0.001)  # 0.2pp to 5pp in 0.1pp steps
power_values = []

for mde in mde_values:
    pow_val = power(total_a, total_b, p_control, min_detectable_diff=mde, alpha=0.05)
    power_values.append(pow_val)

# Create power curve plot
plt.figure(figsize=(10, 6))
plt.plot(mde_values * 100, power_values, linewidth=2)
plt.axhline(y=0.8, color='r', linestyle='--', label='80% Power (standard)')
plt.xlabel('Minimum Detectable Effect (MDE) [percentage points]', fontsize=12)
plt.ylabel('Statistical Power', fontsize=12)
plt.title('Power Analysis: Statistical Power vs Minimum Detectable Effect', fontsize=14)
plt.grid(True metaphor, alpha=0.3)
plt.legend()
plt.xlim(0, 5)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

print(f"\nCurrent sample size: {total_a + total_b:,} total observations")
print(f"Control conversion rate: {p_control:.4f} ({p_control*100:.2f}%)")
print(f"\nPower to detect observed lift ({results['lift']*100:.2f} pp): {power(total_a, total_b, p_control, min_detectable_diff=abs(results['lift']), alpha=0.05):.2%}")

